<a href="https://colab.research.google.com/github/tomonari-masada/course2021-nlp/blob/main/11_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# トピックモデリングの実践

* トピックモデリングを、NMF(nonnegative matrix factorization)とLDA(latent Dirichlet allocation)とで実践
 * LDAの理屈については「統計モデリング２」で。
 * いずれもsklearnの実装を使う。
 * 各トピックの上位単語はワードクラウドで可視化する。

* 入力データは、各文書における各単語の出現回数、またはTF-IDF
 * NMFやLDAはbag-of-wordsモデルなので、語順は考慮されない。
 * LDAの入力データとしては、出現回数を使う。
 * NMFの入力データは、出現回数でも、TF-IDFでも、どちらでも良い。

* 参考資料
 * https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html

* トピックモデルの使い方
 * 文書ベクトルの次元圧縮のための手法としては、いまひとつ性能が良くないかも。
 * EDA (exploratory data analysis) の手法として使うのが良いかも。

## LDAの可視化ツールを先にインストール
* pyLDAvisというツールをインストールすると、ランタイムの再起動が必要になるため

In [ ]:
!pip install pyLDAvis

## データセットの準備
* NeurIPSで発表された1,740本の論文の本文を使う

### データをダウンロードしリスト化する関数を定義

In [ ]:
import io
import os.path
import re
import tarfile
import smart_open


PATH = '/content/drive/MyDrive/data'


def extract_documents(url='https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz'):

  fname = os.path.join(PATH, url.split('/')[-1])

  if not os.path.isfile(fname):
    with smart_open.open(url, "rb") as fin:
      with smart_open.open(fname, 'wb') as fout:
        while True:
          buf = fin.read(io.DEFAULT_BUFFER_SIZE)
          if not buf:
            break
          fout.write(buf)

  with tarfile.open(fname, mode='r:gz') as tar:
  # Ignore directory entries, as well as files like README, etc.
    files = [
             m for m in tar.getmembers()
             if m.isfile() and re.search(r'nipstxt/nips\d+/\d+\.txt', m.name)
             ]
    for member in sorted(files, key=lambda x: x.name):
      member_bytes = tar.extractfile(member).read()
      yield member_bytes.decode('utf-8', errors='replace')

* 実際にデータを取得しリスト化する

In [ ]:
docs = list(extract_documents())

* 文書数、具体的な文書の内容などを確認

In [ ]:
print(len(docs))

In [ ]:
print(docs[0][:1000])

### spaCyを使ってtokenizeする

* 前処理の高速化のため、taggerなどは無効にしておく

In [ ]:
import spacy

nlp = spacy.load('en', disable=["tagger", "parser", "ner"])

* 小文字にしてからtokenizeする関数の定義

In [ ]:
def spacy_lemmatize_text(nlp, text):
  text = nlp(text.lower())
  doc = [word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text]
  return [word for word in doc if len(word) > 1] # 長さ1の単語は削除

* tokenizationの実行

In [ ]:
from tqdm import tqdm

new_docs = list()
for doc in tqdm(docs):
  new_docs.append(spacy_lemmatize_text(nlp, doc))

* tokenizationの結果を確認

In [ ]:
print(new_docs[0])

* 各文書を長い一本の文字列で表現
 * CountVectorizerを後で使うため、トークンを半角スペースでつないだ長い文字列で表しなおす。

In [ ]:
corpus = [' '.join(doc) for doc in new_docs]

In [ ]:
corpus[0]

## データ行列の作成
* NMFの場合、TF-IDFで単語の重みを求めて各文書をベクトル化する。
* LDAの場合、単語の出現頻度をそのまま使って各文書をベクトル化する。

### sklearnのCountVectorizerで疎行列化する

* 全文書の半分より多い文書に現れる単語は、高頻度語とみなして削除する。
* 10件未満の文書にしか現れない単語は、低頻度語とみなして削除する。
* stop wordsも削除する。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.5, min_df=10, stop_words='english')
X = vectorizer.fit_transform(corpus)

In [ ]:
print(X[0])

In [ ]:
print(vectorizer.get_feature_names_out())

In [ ]:
print(len(vectorizer.get_feature_names_out()))

* 文書数と語彙サイズを変数にセット

In [ ]:
X.shape

In [ ]:
n_samples, n_features = X.shape

### TF-IDFで各文書における単語の重みを計算する
* これはNMFの方だけで使う

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
Xtfidf = tfidf.fit_transform(X)

In [ ]:
print(Xtfidf[0])

In [ ]:
Xtfidf.shape

* 抽出するトピックの個数は、今回は20個とする。

In [ ]:
n_components = 20

## NMFでトピック抽出
* まず、TF-IDFのデータ行列を使って　NMFによってトピック抽出を試みる。
 * NMFのパラメータ群は下記サンプルコードのまま。
 * https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py

### NMFとLDAのインポート

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

### NMFによるトピック抽出の実行

In [ ]:
from time import time

print((f"Fitting the NMF model (generalized Kullback-Leibler "
  f"divergence) with tf-idf features, n_samples={n_samples} "
  f"and n_features={n_features}"))
t0 = time()
nmf = NMF(n_components=n_components,
          random_state=12345,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, 
          alpha=.1,
          l1_ratio=.5,
          verbose=1)
nmf.fit(Xtfidf)
print(f"done in {time() - t0:0.3f}s.")

* scikit-learn 1.2以降は、下のように書かないと動かなくなるらしい
 * alphaの代わりにalpha_Wとalpha_Hを使う。
 * ただし、alphaの設定で使った値を、alpha_Wについては特徴量の個数で、alpha_Hについてはインスタンス数で割らないと、元の使い方と同じ結果にならない。

In [ ]:
print((f"Fitting the NMF model (generalized Kullback-Leibler "
  f"divergence) with tf-idf features, n_samples={n_samples} "
  f"and n_features={n_features}"))
t0 = time()
nmf = NMF(n_components=n_components,
          random_state=12345,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000,
          alpha_W=.1 / n_features,
          alpha_H=.1 / n_samples,
          l1_ratio=.5,
          verbose=1)
nmf.fit(Xtfidf)
print(f"done in {time() - t0:0.3f}s.")

* NMFにおける各コンポーネントは、それぞれのトピックにおける単語の重要度を表すベクトルとして表現されている。

In [ ]:
nmf.components_

### トピックの重要語を取り出す関数の定義

In [ ]:
def get_top_words(model, feature_names, n_top_words=30):
  top_features = []
  weights = []
  for topic_idx, topic in enumerate(model.components_):
    top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
    top_features.append([feature_names[i] for i in top_features_ind])
    weights.append(topic[top_features_ind])
  return top_features, weights

### NMFの各コンポーネントから重要語を取り出す

In [ ]:
top_words, weights = get_top_words(nmf, vectorizer.get_feature_names_out())

In [ ]:
print(top_words[0])

In [ ]:
topic_words = [dict(zip(top_words[i], weights[i])) for i in range(n_components)]

In [ ]:
topic_words[0]

### 重要語をワードクラウドで可視化

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
%config InlineBackend.figure_format = 'retina'

* ワードクラウドから除去するストップワードを確認する。

In [ ]:
print(STOPWORDS)

* ワードクラウドを描画

In [ ]:
cloud = WordCloud(stopwords=STOPWORDS,
                  background_color='white',
                  width=1500,
                  height=1000,
                  max_words=100,
                  colormap='tab10'
                  )

In [ ]:
fig, axes = plt.subplots(5, 4, figsize=(16, 25), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
  fig.add_subplot(ax)
  cloud.generate_from_frequencies(topic_words[i], max_font_size=300)
  plt.gca().imshow(cloud)
  plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
  plt.gca().axis('off')

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()

* トピックモデルを使って上のようにワードクラウドを描くと・・・
 * コーパス全体に対して、下のようにたった一つだけ、ワードクラウドを描くことの大雑把さに気づくかも。

In [ ]:
words = dict(zip(vectorizer.get_feature_names_out(), Xtfidf.toarray().sum(0)))
cloud.generate_from_frequencies(words, max_font_size=300)
plt.imshow(cloud);

## LDAでトピック抽出

### LDAによるトピック抽出の実行
* scikit-learnの実装を使う。
* `topic_word_prior`と

In [ ]:
lda = LatentDirichletAllocation(n_components=n_components, 
                                max_iter=20,
                                doc_topic_prior=0.05,
                                topic_word_prior=0.01,
                                learning_method='online',
                                learning_offset=50,
                                batch_size=200,
                                mean_change_tol=1e-4,
                                random_state=12345,
                                evaluate_every=1,
                                verbose=1)

* 入力データは各文書における各単語の出現回数
 * TF-IDFのような、出現回数を加工したデータを使うと、LDAというモデルの構成に合わない。

In [ ]:
print((f"Fitting LDA models with tf features, "
  f"n_samples={n_samples} and n_features={n_features}"))
t0 = time()
lda.fit(X)
print(f"done in {time() - t0:0.3f}s.")

### LDAの各トピックから高確率語を取り出す

In [ ]:
top_words, weights = get_top_words(lda, vectorizer.get_feature_names_out())

In [ ]:
print(top_words[0])

In [ ]:
topic_words = [dict(zip(top_words[i], weights[i])) for i in range(n_components)]

### 高確率語をワードクラウドで可視化

In [ ]:
cloud = WordCloud(stopwords=STOPWORDS,
                  background_color='white',
                  width=1500,
                  height=1000,
                  max_words=100,
                  colormap='tab10'
                  )

In [ ]:
fig, axes = plt.subplots(5, 4, figsize=(16, 25), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
  fig.add_subplot(ax)
  cloud.generate_from_frequencies(topic_words[i], max_font_size=300)
  plt.gca().imshow(cloud)
  plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
  plt.gca().axis('off')

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()

## LDAの可視化ツールpyLDAvisを使う

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, X, vectorizer, mds='tsne')
panel